# Algorithms with Matrices

In this practice we are going to implement algorithms to work with matrices:
- solving simple equations
- inverting matrices
- decomposing matrices to LU decompositions

For these activities Gauss Elimination is our main tool. We will only work with matrices that do not require swapping the rows.

We will also perform QR decomposition using Gramm-Schmidt orthonormalization method

In [ ]:
import numpy as np
import lovely_numpy as ln
import json_tricks 
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(0)
# --placeholder start --

inputs = []
for index in range(100):
    N = np.random.randint(1, 10)
    L = np.random.randint(size=(N, N), low=-10, high=10)
    L = np.tril(L)
    U = np.random.randint(size=(N, N), low=-10, high=10)
    U = np.triu(U)
    np.fill_diagonal(L, 1)
    np.fill_diagonal(U, 1)
    A = L @ U

    x = np.random.randint(size=(N,), low=-10, high=10)
    b = A @ x
    inputs.append({'A': A, 'b': b})


import os
os.makedirs('inputs', exist_ok=True)
json_tricks.dump(inputs, 'inputs/inputs.json')
# --placeholder end --

inputs = json_tricks.load('inputs/inputs.json')
answer = {}

# Task 1. Implement Gauss Elimination Stage I

Your task here is to create an algorithm that will perfrom mutations of matrix $A$ that transform it to Upper-Triangular form $U$.

$$
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1N} \\
a_{21} & a_{22} & \dots & a_{2N} \\
\vdots & \vdots & \ddots & \vdots \\
a_{N1} & a_{N2} & \dots & a_{NN}
\end{bmatrix}

\rightarrow

\begin{bmatrix}
1 & a_{12}^* & \dots & a_{1N}^* \\
0 & 1 & \dots & a_{2N}^* \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
$$

Note that there should be unit diagonal after the 1 stage.

The same mutations for rows as you do to the matrix $A$ should be done to the rows of matrix $B$. 

What can matrix $B$ be? It can be:
- vector of right hand side $b$ (in that case we are solving the System of Linear Equations)
- identity matrix (in that case we invert matrix)
- actually, Stage I can be used as Stage II, and in that case right hand matrix can be some matrix from Stage I


In [ ]:
import numpy as np

def gauss_elimination(A, B):
    A = A.copy().astype(float)
    B = B.copy().astype(float)

    for index in range(A.shape[1]):
        for row in range(index + 1, A.shape[0]):
            factor = A[row, index] / A[index, index]
            A[row, index:] -= factor * A[index, index:]
            B[row] -= factor * B[index]
        
        factor = A[index, index]
        A[index] /= factor
        B[index] /= factor
    
    return A, B


In [ ]:
answer['gauss_elimination_1'] = [
    gauss_elimination(A=input['A'], B=input['b']) for input in inputs
]

# Task 2: Solve System of Linear Equations (SLE)

For the SLE given by matrix $A$ and right-hand vector $\mathbf b$, implement a function that solves this SLE

You will need to use twice Gauss Elimination Algorithm

1. Apply it to matrix $A$ and to vector $\mathbf b$ as matrix $B$ as Gauss Elimintaion I stage
2. For Gauss Elimination II stage you can use the same algorithm by just applying it to reversed matrix 

    (by enumerating equations and variables in reversed order, we can rewrite the matrix $U$ in low-triangle form $L$). 

    Something has to be done to the right-hand side vector too
3. Form the answer from output of Gauss Elimination Stage II

In [ ]:
import numpy as np

def solve_equation(A, b):
    res = None

    A1, b1 = gauss_elimination(A, b)

    A2, b2 = gauss_elimination(A1[::-1, ::-1], b1[::-1])

    res = b2[::-1]
    
    return res


You can always check that your solution is correct by subtracting

$A \mathbf x - \mathbf b$, which should give you $\mathbf 0$

By the way, matrix product, such as $A \mathbf x$ in numpy is done using `A @ x` operation

In [ ]:
answer['solutions'] = [
    solve_equation(input['A'], input['b']) for input in inputs
]

print(inputs[0]['A'] @ answer['solutions'][0] - inputs[0]['b'])


# Task 3: Find inverse of $A$

Use the same approach as with solving SLE, but now instead of $\mathbf b$ for right hand side, insert corresponding Identity matrix $I$ as input into Gaussian Elimination algorithm

Identitiy matrix of size $N \times N$ is built using `numpy.eye(N)` 

In [ ]:
import numpy as np

def find_inverse(A):
    res = None

    # Apply Gauss Elimination to A with identity matrix as B
    A1, B1 = gauss_elimination(A, np.eye(A.shape[0]))

    # Apply Gauss Elimination to the reversed A1 and B1 matrices
    A2, B2 = gauss_elimination(A1[::-1, ::-1], B1[::-1, ::-1])

    # The result is the reversed B2 matrix, which is the inverse of A
    res = B2[::-1, ::-1]
    
    return res

You can always check that the matrix that you have found is indeed inverse to $A$ as $A^{-1} A = I$

In [ ]:
answer['inverses'] = [
    find_inverse(input['A']) for input in inputs
]

# Check if A * A_inv = I
print(answer['inverses'][0] @ inputs[0]['A'])


# Task 4: LU decomposition

You can find LU decomposition of the matrix $A$ using Gauss Elimination. Again, it is done using 2 stages of Gauss Elimination:

1. $A \mathbf x = I \mathbf b \rightarrow U \mathbf x = L^* \mathbf b$

2. $I U \mathbf x = L^* \mathbf b \rightarrow L U \mathbf x = I \mathbf b$

In [ ]:
import numpy as np

def find_lu(A):
    L, U = None, None

    # Apply Gauss Elimination to A and identity matrix to get U and L
    U1, L1 = gauss_elimination(A, np.eye(A.shape[0]))

    # Apply Gauss Elimination to L1 to get the final L matrix
    I2, L2 = gauss_elimination(L1, np.eye(A.shape[0]))

    # The result is L and U matrices
    L = L2
    U = U1
    
    return L, U

You can always check that your LU decomposition works as

$LU - A = O$ (a zero matrix)

In [ ]:
answer['LU'] = [
    find_lu(input['A']) for input in inputs
]

# Check if LU = A
print(answer['LU'][0][0] @ answer['LU'][0][1] - inputs[0]['A'])


# QR decomposition

Now again we will consider matrix $A^T$ in context of SLE:

$A^T \mathbf x = I \mathbf b$

Out of matrix $A$ we can make an orthonormal matrix $Q$ by orthonoramlizing the rows, this process will turn the $I$ matrix on the right to lower-triangular matrix $L^*$:

$A^T \mathbf x = I \mathbf b \rightarrow Q^T \mathbf x = L^* \mathbf b$

After that we should invert matrix $L^*$ and we will get:

$I Q^T \mathbf x = L^* \mathbf b \rightarrow L Q^T \mathbf x = \mathbf b$

If we transpose that matrix, we will get  QR decomposition of matrix $A$:

$A = Q R,$

where $R$ is upper-triangular matrix.

The task is to:
1. Implement Gramm-Schmidt orthonormalization procedure
2. Combining Gramm-Schmidt algorithm and Gaussian Algorithm, implement QR decomposition

# Task 5: Gramm-Schmidt Algorithm for Matrices

Implement Gramm-Schmidt process for **rows** of matrix $A$ (and do the same transforms to matrix $B$)

In [ ]:
import numpy as np

def gramm_schmidt(A, B):
    A = A.copy().astype(float)
    B = B.copy().astype(float)

    # Perform Gramm-Schmidt process for each row
    for i in range(A.shape[1]):
        for j in range(i):
            factor = np.dot(A[i, :], A[j, :])
            A[i, :] -= factor * A[j, :]
            B[i, :] -= factor * B[j, :]
        
        # Normalize the row
        factor = np.linalg.norm(A[i, :])
        A[i, :] /= factor
        B[i, :] /= factor
    
    return A, B


In [ ]:
answer['gramm_schmidt'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs
]


print("===== Should be close to 1s: =====")
print((answer['gramm_schmidt'][0][0] ** 2).sum(axis=1))
print()
print("===== Should be close to L: =====")
print(answer['gramm_schmidt'][0][1])


# Task 6: QR decomposition

Implement QR decomposition algorithm. Note that we work on rows on matrix $A$ with our Gauss and Gramm-Schmidt algorithms, but QR decomposition is defined for column matrix 

In [ ]:
import numpy as np

def find_qr(A):
    B = np.eye(A.T.shape[0])

    # Apply Gramm-Schmidt process on A.T and B
    QT, L1 = gramm_schmidt(A.T, B)

    # Apply Gauss Elimination to L1 to obtain R
    I, LT = gauss_elimination(L1, np.eye(A.shape[0]))

    # The result is Q and R
    Q = QT.T
    R = LT.T
    
    return Q, R


In [ ]:
answer['QR'] = [
    find_qr(input['A']) for input in inputs
]

# Check if Q * R = A
print(answer['QR'][0][0] @ answer['QR'][0][1] - inputs[0]['A'])


In [ ]:
json_tricks.dump(answer, '.answer.json')